In [104]:
# Import necessary libraries
import logging
import sqlite3
import pandas as pd
import warnings
import numpy as np
import matplotlib.pyplot as plt
import rpy2.robjects as robjects
from datetime import datetime
from rpy2.robjects import pandas2ri
from rpy2.robjects import pandas2ri, r, globalenv, StrVector, conversion

In [105]:
# Enable pandas to R dataframe conversion
pandas2ri.activate()

In [106]:
# Setting display to view all columns
pd.set_option('display.max_columns', 40)

In [107]:
# Ignore Warnings
warnings.filterwarnings("ignore")

In [108]:
class InfoFilter(logging.Filter):
    def filter(self, record):
        return record.levelno == logging.INFO

In [109]:
logging.basicConfig(
    filename='logs/results_log.log',
    filemode='a',  # Append mode to add new entries without deleting old ones
    format='%(asctime)s - %(levelname)s - %(message)s',
    level=logging.INFO  # Log messages of level INFO and above
)
# Get the logger object and add a filter to only accept INFO messages
logger = logging.getLogger()
info_filter = InfoFilter()
logger.addFilter(info_filter)

In [110]:
# Set State, County, MSA Code, and Stat Code
statename = StrVector(["NJ"])
countyname = StrVector(["Middlesex"])
MSACODE = 35620
STATECODE = 34

In [111]:
def filter_data(MSACODE,STATECODE,ind_data):
    twday_flag=None
    twday_name=None
    gender_name=None
    # Filtering dataset based on MSA and STATE
    if MSACODE is not None:
        ind_data = ind_data[(ind_data['EST_MSA'] == MSACODE) & (ind_data['EST_ST'] == STATECODE)]
    else:
        ind_data = ind_data[(ind_data['EST_ST'] == STATECODE)]
    # Calculating age based on the year the survey was conducted in
    ind_data['AGE'] = 2024 - ind_data['TBIRTH_YEAR']
    if 'TW_START' in ind_data.columns and 'EGENDER' in ind_data.columns:
        print(ind_data.head())
        # Filtering dataset based on variables of interest
        variables_of_interest = ["EST_ST","EST_MSA", "MS", "RRACE", "TW_START", "EEDUC", "KINDWORK", "THHLD_NUMPER", "INCOME", "EGENDER", "AGE", "RHISPANIC", "ANYWORK"]
        ind_data = ind_data[variables_of_interest]
        twday_flag = 1
    elif 'TW_YN' in ind_data.columns and 'EGENDER' in ind_data.columns:
        print(ind_data.head())
        # Filtering dataset based on variables of interest
        variables_of_interest = ["EST_ST","EST_MSA", "MS", "RRACE", "TW_YN", "EEDUC", "KINDWORK", "THHLD_NUMPER", "INCOME", "EGENDER", "AGE", "RHISPANIC", "ANYWORK"]
        ind_data = ind_data[variables_of_interest]
        twday_flag = 2
    elif 'ACTIVITY2' in ind_data.columns and 'EGENID_BIRTH' in ind_data.columns:
        print(ind_data.head())
        # Filtering dataset based on variables of interest
        variables_of_interest = ["EST_ST","EST_MSA", "MS", "RRACE", "ACTIVITY2", "EEDUC", "KINDWORK", "THHLD_NUMPER", "INCOME", "EGENID_BIRTH", "AGE", "RHISPANIC", "ANYWORK"]
        ind_data = ind_data[variables_of_interest]
        twday_flag = 3
    else: 
        # Filtering dataset based on variables of interest
        print(ind_data.head())
        variables_of_interest = ["EST_ST","EST_MSA", "MS", "RRACE", "TWDAYS", "EEDUC", "KINDWORK", "THHLD_NUMPER", "INCOME", "EGENID_BIRTH", "AGE", "RHISPANIC", "ANYWORK"]
        ind_data = ind_data[variables_of_interest]
        twday_flag = 4

    if twday_flag==1:
        twday_name='TW_START'
        gender_name='EGENDER'
    elif twday_flag==2:
        twday_name='TW_YN'
        gender_name='EGENDER'
    elif twday_flag==3:
        twday_name='ACTIVITY2'
        gender_name='EGENID_BIRTH'
    else:
        twday_name='TWDAYS'
        gender_name='EGENID_BIRTH'

    # Checking for unique occurrences for chosen constraints
    print(sorted(ind_data['AGE'].unique()))
    print(sorted(ind_data['EEDUC'].unique()))
    print(sorted(ind_data[gender_name].unique()))
    print(sorted(ind_data['KINDWORK'].unique()))
    print(sorted(ind_data['THHLD_NUMPER'].unique()))
    print(sorted(ind_data[twday_name].unique()))

    # Replacing -99 and -88 codes with NA to treat them as missing
    ind_data['EEDUC'].replace([-99, -88], pd.NA, inplace=True)
    ind_data[twday_name].replace([-99, -88], pd.NA, inplace=True)
    ind_data['KINDWORK'].replace([-99, -88], pd.NA, inplace=True)
    # Dropping rows with NA values, and filtering data to keep individuals aged 25 and above, and who are employed
    ind_data.dropna(inplace=True)
    ind_data = ind_data[ind_data['AGE'] >= 25]
    ind_data = ind_data[ind_data['ANYWORK'] == 1]
    # Checking for unique occurrences for chosen constraints after filtering
    print("After filtering:")
    print(sorted(ind_data['AGE'].unique()))
    print(sorted(ind_data['EEDUC'].unique()))
    print(sorted(ind_data[gender_name].unique()))
    print(sorted(ind_data['KINDWORK'].unique()))
    print(sorted(ind_data['THHLD_NUMPER'].unique()))
    print(sorted(ind_data[twday_name].unique()))
    # Fixed arrays
    brks = [25, 30, 35, 40, 45, 50, 55, 60, 62, 65, 67, 70, 75, 80, 85, float('inf')]
    labs = [
    "25 to 29 years", "30 to 34 years", "35 to 39 years", "40 to 44 years", "45 to 49 years",
    "50 to 54 years", "55 to 59 years", "60 and 61 years", "62 to 64 years", "65 and 66 years",
    "67 to 69 years", "70 to 74 years", "75 to 79 years", "80 to 84 years", "85 years and over"
    ]
    # Input unique values
    unique_values = ind_data['AGE'].unique()
    # Convert unique_values to a set for faster lookup
    unique_values_set = set(unique_values)
    # Age category dictionary
    age_cat_dict = {
    "25 to 29 years": [25, 26, 27, 28, 29],
    "30 to 34 years": [30, 31, 32, 33, 34],
    "35 to 39 years": [35, 36, 37, 38, 39],
    "40 to 44 years": [40, 41, 42, 43, 44],
    "45 to 49 years": [45, 46, 47, 48, 49],
    "50 to 54 years": [50, 51, 52, 53, 54],
    "55 to 59 years": [55, 56, 57, 58, 59],
    "60 and 61 years": [60, 61],
    "62 to 64 years": [62, 63, 64],
    "65 and 66 years": [65, 66],
    "67 to 69 years": [67, 68, 69],
    "70 to 74 years": [70, 71, 72, 73, 74],
    "75 to 79 years": [75, 76, 77, 78, 79],
    "80 to 84 years": [80, 81, 82, 83, 84],
    "85 years and over": list(range(85, 200))
    }
    # Create new bins and labels based on the presence of unique values
    new_brks = [brks[0]]
    new_labs = []
    for label, ages in age_cat_dict.items():
        # Check if there are any values in the current age range
        if any(age in unique_values_set for age in ages):
            # Append the end value of the current bin
            if label != "85 years and over":
                new_brks.append(brks[len(new_labs) + 1])
            else:
                new_brks.append(float('inf'))
            new_labs.append(label)
    # Output the updated bins and labels
    print("Updated Bins:", new_brks)
    print("Updated Labels:", new_labs)
    # Encoding/ recategorizing constraints
    ind_data['AGE'] = pd.cut(ind_data['AGE'], bins=new_brks, labels=new_labs, right=False)
    ind_data[gender_name] = ind_data[gender_name].replace({1: 'Male', 2: 'Female'})
    ind_data['EEDUC'] = ind_data['EEDUC'].replace({1: 'Less than high school',2: 'Some high school', 3: 'High school graduate or equivalent (for example GED)', 4: "Some college, but degree not received or is in progress", 5: "Associate's degree (for example AA, AS)", 6: "Bachelor's degree (for example BA, BS, AB)", 7: "Graduate degree (for example master's, professional, doctorate)"})
    ind_data['KINDWORK'] = ind_data['KINDWORK'].replace({1: 'Government', 2: 'Private company', 3: 'Non-profit organization including tax exempt and charitable organizations', 4: 'Self-employed', 5: 'Working in a family business'})
    if twday_flag==1:    
        ind_data[twday_name] = ind_data[twday_name].replace({1:"Yes, at least one adult substituted some or all of their typical in-person work for telework",2:"No, no adults substituted their typical in-person work for telework",3:"No, there has been no change in telework"})
    elif twday_flag==2 or twday_flag==3:
        ind_data[twday_name] = ind_data[twday_name].replace({1:"Yes", 2:"No"})
    else:
        ind_data[twday_name] = ind_data[twday_name].replace({1: 'Yes, for 1-2 days', 2: 'Yes, for 3-4 days', 3: 'Yes, for 5 or more days', 4: 'No'})
    ind_data['INCOME'] = ind_data['INCOME'].replace({1: 'Less than $25,000', 2: '$25,000 - $34,999', 3: '$35,000 - $49,999', 4: '$50,000 - $74,999', 5: '$75,000 - $99,999', 6: '$100,000 - $149,999', 7: '$150,000 - $199,999', 8: '$200,000 and above'})
    ind_data['MS'] = ind_data['MS'].replace({1: 'Now married', 2: 'Widowed', 3: 'Divorced', 4: 'Separated', 5: 'Never married'})
    # Creating subsets of the data
    ind_full = ind_data[[gender_name, "AGE", "EEDUC", "KINDWORK", "INCOME", "MS", "THHLD_NUMPER", twday_name]]
    ind_data = ind_data[["EST_ST", gender_name, "AGE", "EEDUC", "KINDWORK", "MS", "THHLD_NUMPER", "INCOME", twday_name]]
    # Checking for unique occurrences for chosen constraints after recategorizing
    print(sorted(ind_data['AGE'].unique()))
    print(sorted(ind_data['EEDUC'].unique()))
    print(sorted(ind_data[gender_name].unique()))
    print(sorted(ind_data['KINDWORK'].unique()))
    print(sorted(ind_data[twday_name].unique()))
    edu_labs = ind_data['EEDUC'].unique()
    print(edu_labs)
    work_labs = ind_data['KINDWORK'].unique()
    print(work_labs)
    return ind_data,ind_full,new_brks,new_labs,edu_labs,work_labs,gender_name,twday_name


In [112]:
def con_age_func(new_brks,new_labs,statename,countyname):
  # Load the R script containing the function
  r.source('age_sexcons.R')  # Replace 'age_sexcons.R' with the path to your R script
  # Reference the function from the R script
  process_acs_age_sex_data = globalenv['process_acs_age_sex_data']
  # Call the R function from Python
  result = process_acs_age_sex_data(new_brks, new_labs, statename, countyname)
  # Since the result is expected to be a data.frame, convert it to Pandas DataFrame using conversion context
  with conversion.localconverter(robjects.default_converter + pandas2ri.converter):
    con_age_sex_df = robjects.conversion.rpy2py(result)
  # Display the result
  con_age = con_age_sex_df['con_age']
  con_sex = con_age_sex_df['con_sex']
  con_age_sex_zero_row = con_age_sex_df['zero_row_indices']
  print(con_age.head())
  print(con_sex.head())
  print(con_age_sex_zero_row)
  return con_age, con_sex

In [113]:
def con_edu_func(edu_labs,statename,countyname):
  r.source("educons.R")
  process_edu_data = globalenv["process_edu_data"]
  result = process_edu_data(edu_labs,statename,countyname)
  # Since the result is expected to be a data.frame, convert it to Pandas DataFrame using conversion context
  with conversion.localconverter(robjects.default_converter + pandas2ri.converter):
    con_edu_df = robjects.conversion.rpy2py(result)

  # Display the result
  con_edu = con_edu_df['con_edu']
  con_edu_zero_row = con_edu_df['zero_row_indices']
  print(con_edu.head())
  print(con_edu_zero_row)
  return con_edu


In [114]:
def con_work_func(work_labs,statename,countyname):
  r.source("kindworkcons.R")
  process_work_data = globalenv["process_work_data"]
  result = process_work_data(work_labs,statename,countyname)
  # Since the result is expected to be a data.frame, convert it to Pandas DataFrame using conversion context
  with conversion.localconverter(robjects.default_converter + pandas2ri.converter):
    con_work_df = robjects.conversion.rpy2py(result)

  # Display the result
  con_work = con_work_df['con_work']
  con_work_zero_row = con_work_df['zero_row_indices']
  print(con_work.head())
  print(con_work_zero_row)
  return con_work

In [115]:
def constraint_build(con_age,con_sex,con_edu,con_work):
    # Checking the Total sums of each constraint
    sum_con_age = con_age.sum(numeric_only=True).sum()
    sum_con_sex = con_sex.sum(numeric_only=True).sum()
    sum_con_edu = con_edu.sum(numeric_only=True).sum()
    sum_con_work = con_work.sum(numeric_only=True).sum()
    print("Sum of age:{}".format(sum_con_age))
    print("Sum of sex:{}.".format(sum_con_sex))
    print("Sum of education:{}".format(sum_con_edu))
    print("Sum of work:{}".format(sum_con_work))
    # List of dataframes
    dataframes = [con_age, con_sex, con_edu, con_work]
    # Step 1: Find common GEOID values across all dataframes
    common_geoids = set(dataframes[0]['GEOID'])
    for df in dataframes[1:]:
        common_geoids.intersection_update(df['GEOID'])
    # Step 2: Filter each dataframe to keep only the rows with common GEOID values
    filtered_dataframes = [df[df['GEOID'].isin(common_geoids)] for df in dataframes]
    # Assign the filtered dataframes back to the original variables
    con_age, con_sex, con_edu ,con_work = filtered_dataframes
    geoids = con_age[['GEOID']]
    print(geoids)
    con_age = con_age.drop(con_age.columns[0], axis=1)
    con_sex = con_sex.drop(con_sex.columns[0], axis=1)
    con_edu = con_edu.drop(con_edu.columns[0], axis=1)
    con_work = con_work.drop(con_work.columns[0], axis=1)
    # Checking the Total sums of each constraint
    sum_con_age = con_age.sum(numeric_only=True).sum()
    sum_con_sex = con_sex.sum(numeric_only=True).sum()
    sum_con_edu = con_edu.sum(numeric_only=True).sum()
    sum_con_work = con_work.sum(numeric_only=True).sum()
    print("Sum of age:{}".format(sum_con_age))
    print("Sum of sex:{}.".format(sum_con_sex))
    print("Sum of education:{}".format(sum_con_edu))
    print("Sum of work:{}".format(sum_con_work))
    con_age = con_age.reset_index(drop=True)
    con_sex = con_sex.reset_index(drop=True)
    con_edu = con_edu.reset_index(drop=True)
    con_work = con_work.reset_index(drop=True)
    # Compute total minimum attribute counts for each row
    # min_con = min(con_age,con_sex,con_edu,con_work)
    total_income_counts = con_work.sum(axis=1)
    print(total_income_counts)
    # Define the adjustment and correction function
    def adjust_and_correct(df, target_totals):
        # Compute the initial proportions
        proportions = df.div(df.sum(axis=1), axis=0)
        
        # Apply target totals using proportions
        adjusted_values = proportions.mul(target_totals, axis=0)
        rounded_values = adjusted_values.round()
        
        # Efficiently correct residuals
        residuals = target_totals - rounded_values.sum(axis=1)
        for idx in residuals[residuals != 0].index:
            sign = np.sign(residuals[idx])
            row_proportions = proportions.loc[idx]
            sorted_indices = row_proportions.sort_values(ascending=False).index
            for col in sorted_indices:
                if sign == 1 and residuals[idx] > 0:
                    max_add = target_totals[idx] - rounded_values.loc[idx].sum()
                    add_value = min(max_add, residuals[idx])
                    rounded_values.loc[idx, col] += add_value
                    residuals[idx] -= add_value
                elif sign == -1 and residuals[idx] < 0:
                    max_sub = rounded_values.loc[idx, col]
                    sub_value = min(max_sub, -residuals[idx])
                    rounded_values.loc[idx, col] -= sub_value
                    residuals[idx] += sub_value
                if residuals[idx] == 0:
                    break
        
        # Validation (optional but recommended)
        validation_passed = (rounded_values.sum(axis=1) == target_totals).all()
        return {
            'rounded_values': rounded_values,
            'validation_passed': validation_passed
        }

    # Apply the function to your datasets
    results_edu = adjust_and_correct(con_edu, total_income_counts)
    results_age = adjust_and_correct(con_age, total_income_counts)
    results_sex = adjust_and_correct(con_sex, total_income_counts)

    # Output the results to check
    print("Adjusted and Corrected Education Data:")
    print(results_edu['rounded_values'])
    print("\nAdjusted and Corrected Age Data:")
    print(results_age['rounded_values'])
    print("\nAdjusted and Corrected Sex Data:")
    print(results_sex['rounded_values'])
    # Assign corrected values back to the original dataframes
    con_age = results_age['rounded_values']
    con_sex = results_sex['rounded_values']
    con_edu = results_edu['rounded_values']
    # con_work = results_work['rounded_values']

    # Sum the total values of the different corrected constraints
    sum_con_age = con_age.values.sum()
    sum_con_sex = con_sex.values.sum()
    sum_con_edu = con_edu.values.sum()
    sum_con_work = con_work.values.sum()

    # Display the total sum of corrected constraints
    print("Total Sum of Corrected Age Constraints:")
    print(sum_con_age)
    print("\nTotal Sum of Corrected Sex Constraints:")
    print(sum_con_sex)
    print("\nTotal Sum of Corrected Education Constraints:")
    print(sum_con_edu)
    print("\nTotal Sum of Corrected Work Constraints:")
    print(sum_con_work)
    logger.info(f"Sum of all Constraints: {sum_con_age}")

    # Compare row sums of different corrected constraints
    print("\nRow sums comparison between corrected constraints:")
    print((con_work.sum(axis=1) == con_age.sum(axis=1)).all())
    print((con_work.sum(axis=1) == con_sex.sum(axis=1)).all())
    print((con_work.sum(axis=1) == con_work.sum(axis=1)).all())
    print((con_work.sum(axis=1) == con_edu.sum(axis=1)).all())
    # Create constraint dataframe by combining all constraints
    cons = pd.concat([con_age, con_sex, con_edu, con_work], axis=1)
    return cons,con_age,con_sex,con_edu,con_work,geoids

In [116]:
def int_trs(x):
    # Convert input to a flat array
    xv = np.array(x).flatten()
    
    # Integer part of the weight
    xint = np.floor(xv).astype(int)
    
    # Decimal part of the weight
    r = xv - xint
    
    # Deficit population (rounded sum of remaining weights)
    deficit = int(np.round(np.sum(r)))
    
    # Top up the weights (+1 applied)
    if deficit > 0:
        topup_indices = np.random.choice(len(xv), size=deficit, p=r/np.sum(r), replace=False)
        for idx in topup_indices:
            xint[idx] += 1

    # Reshape and set dimnames if needed
    xint = xint.reshape(x.shape)
    
    return xint

# Function 2: Equivalent to `int_expand_vector` in R
def int_expand_vector(x):
    # Create an array of indices
    indices = np.arange(1, len(x) + 1)
    
    # Repeat each index according to the rounded weight
    expanded_vector = np.repeat(indices, np.round(x).astype(int))
    
    return expanded_vector

In [117]:
def ipfmain(cons,con_age,con_sex,con_edu,con_work,ind_data,ind_full,geoids,gender_name,twday_name):
    # Creating dummy variables for each column with 0 and 1 representation
    cat_age = pd.get_dummies(ind_data['AGE'], prefix='AGE', drop_first=False).astype(int)
    cat_sex = pd.get_dummies(ind_data[gender_name], prefix=gender_name, drop_first=False).astype(int)
    cat_edu = pd.get_dummies(ind_data['EEDUC'], prefix='EEDUC', drop_first=False).astype(int)
    cat_work = pd.get_dummies(ind_data['KINDWORK'], prefix='KINDWORK', drop_first=False).astype(int)

    # Getting the dimensions of each dummy matrix
    print(cat_age.shape)
    print(cat_sex.shape)
    print(cat_edu.shape)
    print(cat_work.shape)
    # Combine the dummy matrices horizontally (similar to `cbind()` in R)
    ind_cat = pd.concat([cat_age, cat_sex, cat_edu, cat_work], axis=1)

    # Display the first few rows of the resulting DataFrame
    ind_cat.head()
    # Calculate column sums of ind_cat
    ind_agg = ind_cat.sum(axis=0)
    ind_agg

    # Number of rows in `cons`
    n_zone = cons.shape[0]
    print(n_zone)

    # Number of rows in `ind_data`
    n_ind = ind_data.shape[0]
    print(n_ind)

    # Number of columns in `con_age`
    n_age = con_age.shape[1]
    print(n_age)

    # Number of columns in `con_sex`
    n_sex = con_sex.shape[1]
    print(n_sex)

    # Number of columns in `con_edu`
    n_edu = con_edu.shape[1]
    print(n_edu)

    # Number of columns in `con_work`
    n_work = con_work.shape[1]
    print(n_work)

    # First Iteration with age constraint
    epsilon = 1e-10  # Small constant to prevent division by zero

    # Step 1: Initialize Weights
    weights = np.ones((n_ind, n_zone))

    # Create independent copies for multiple weights
    weights1 = weights.copy()
    weights2 = weights.copy()
    weights3 = weights.copy()
    weights4 = weights.copy()
    weights5 = weights.copy()
    weights6 = weights.copy()

    print("Weights dimensions:", weights.shape)
    # Step 2: Create `ind_agg0` Equivalent
    ind_agg0 = np.array([ind_agg * 1 for x in cons.values])

    # Create DataFrame for consistency and assign column names
    ind_agg0 = pd.DataFrame(ind_agg0, columns=cons.columns)
    ind_agg0.head()
    
    # Step 3: Update Weights (`weights1`) with Nested Loops
    for j in range(n_zone):
        for i in range(n_age):
            # Find the indices where the corresponding value in `ind_cat` is 1
            index = ind_cat.iloc[:, i] == 1

            # Apply epsilon correction to prevent division by zero
            adjusted_denominator = ind_agg0.iloc[j, i] + epsilon
            if adjusted_denominator <= 0:
                adjusted_denominator = epsilon

            # Update weights1 where the index condition is True
            weights1[index, j] = weights[index, j] * con_age.iloc[j, i] / adjusted_denominator

    weights1_df = pd.DataFrame(weights1)
    weights1_df.head()
    # Step 4: Initialize `ind_agg` Variables with NaN
    ind_agg1 = ind_agg0.copy() * np.nan
    ind_agg2 = ind_agg0.copy() * np.nan
    ind_agg3 = ind_agg0.copy() * np.nan
    ind_agg4 = ind_agg0.copy() * np.nan
    ind_agg5 = ind_agg0.copy() * np.nan
    ind_agg6 = ind_agg0.copy() * np.nan
    # Step 5: Update `ind_agg1` by Summing Conditioned Multiplications
    for i in range(n_zone):
        # Calculate column sums after element-wise multiplication of `ind_cat` and `weights1[:, i]`
        ind_agg1.iloc[i, :] = (ind_cat.values * weights1[:, i].reshape(-1, 1)).sum(axis=0)

    ind_agg1.head()
    # Step 6: Calculate and Compare Row Sums
    # Calculate row sums for columns 0 to 13 (which corresponds to columns 1 to 14 in R)
    row_sums_ind_agg1 = ind_agg1.iloc[:, 0:n_age-1].sum(axis=1)
    row_sums_cons = cons.iloc[:, 0:n_age-1].sum(axis=1)

    # Compare row sums
    row_sums_equal = row_sums_ind_agg1 == row_sums_cons

    # Display row sums and the comparison results
    print("Row sums for ind_agg1:\n", row_sums_ind_agg1)
    print("Row sums for cons:\n", row_sums_cons)
    print("Are row sums equal:\n", row_sums_equal)

    # Second Iteration with Age & Sex constraint
    epsilon = 1e-10  # Small constant to prevent division by zero
    for j in range(n_zone):
        for i in range(n_age+n_sex):
            # Find the indices where the corresponding value in `ind_cat` is 1
            index = ind_cat.iloc[:, i] == 1

            # Apply epsilon correction to prevent division by zero
            adjusted_denominator = ind_agg1.iloc[j, i] + epsilon
            if adjusted_denominator <= 0:
                adjusted_denominator = epsilon

            # Update weights1 where the index condition is True
            weights2[index, j] = weights1[index, j] * cons.iloc[j, i] / adjusted_denominator

    # Convert to DataFrame and display the first few rows
    weights2_df = pd.DataFrame(weights2)
    weights2_df.head()
    for i in range(n_zone):
    # Calculate column sums after element-wise multiplication of `ind_cat` and `weights2[:, i]`
        ind_agg2.iloc[i, :] = (ind_cat.values * weights2[:, i].reshape(-1, 1)).sum(axis=0)

    ind_agg2.head()
    row_sums_ind_agg2 = ind_agg2.iloc[:, n_age-1:(n_age+n_sex)-1].sum(axis=1)
    row_sums_cons = cons.iloc[:, n_age-1:(n_age+n_sex)-1].sum(axis=1)

    # Compare row sums
    row_sums_equal = row_sums_ind_agg2 == row_sums_cons

    # Display row sums and the comparison results
    print("Row sums for ind_agg2:\n", row_sums_ind_agg2)
    print("Row sums for cons:\n", row_sums_cons)
    print("Are row sums equal:\n", row_sums_equal)

    # Third iteration with Age, Sex, and Education constraint
    epsilon = 1e-10  # Small constant to prevent division by zero
    for j in range(n_zone):
        for i in range(n_age+n_sex+n_edu):
            # Find the indices where the corresponding value in `ind_cat` is 1
            index = ind_cat.iloc[:, i] == 1

            # Apply epsilon correction to prevent division by zero
            adjusted_denominator = ind_agg2.iloc[j, i] + epsilon
            if adjusted_denominator <= 0:
                adjusted_denominator = epsilon

            # Update weights1 where the index condition is True
            weights3[index, j] = weights2[index, j] * cons.iloc[j, i] / adjusted_denominator

    # Convert to DataFrame and display the first few rows
    weights3_df = pd.DataFrame(weights3)
    weights3_df.head()
    for i in range(n_zone):
    # Calculate column sums after element-wise multiplication of `ind_cat` and `weights1[:, i]`
        ind_agg3.iloc[i, :] = (ind_cat.values * weights3[:, i].reshape(-1, 1)).sum(axis=0)

    ind_agg3.head()
    row_sums_ind_agg3 = ind_agg3.iloc[:, (n_age+n_sex)-1:(n_age+n_sex+n_edu)-1].sum(axis=1)
    row_sums_cons = cons.iloc[:, (n_age+n_sex)-1:(n_age+n_sex+n_edu)-1].sum(axis=1)

    # Compare row sums
    row_sums_equal = row_sums_ind_agg3 == row_sums_cons

    # Display row sums and the comparison results
    print("Row sums for ind_agg3:\n", row_sums_ind_agg3)
    print("Row sums for cons:\n", row_sums_cons)
    print("Are row sums equal:\n", row_sums_equal)

    # Fourth iteration with Age, Sex, Education, and Work constraint
    epsilon = 1e-10  # Small constant to prevent division by zero
    for j in range(n_zone):
        for i in range(n_age+n_sex+n_edu+n_work):
            # Find the indices where the corresponding value in `ind_cat` is 1
            index = ind_cat.iloc[:, i] == 1

            # Apply epsilon correction to prevent division by zero
            adjusted_denominator = ind_agg3.iloc[j, i] + epsilon
            if adjusted_denominator <= 0:
                adjusted_denominator = epsilon

            # Update weights1 where the index condition is True
            weights4[index, j] = weights3[index, j] * cons.iloc[j, i] / adjusted_denominator

    # Convert to DataFrame and display the first few rows
    weights4_df = pd.DataFrame(weights4)
    weights4_df.head()
    for i in range(n_zone):
    # Calculate column sums after element-wise multiplication of `ind_cat` and `weights1[:, i]`
        ind_agg4.iloc[i, :] = (ind_cat.values * weights4[:, i].reshape(-1, 1)).sum(axis=0)

    ind_agg4.head()
    row_sums_ind_agg4 = ind_agg4.iloc[:, (n_age+n_sex+n_edu)-1:(n_age+n_sex+n_edu+n_work)-1].sum(axis=1)
    row_sums_cons = cons.iloc[:, (n_age+n_sex+n_edu)-1:(n_age+n_sex+n_edu+n_work)-1].sum(axis=1)

    # Compare row sums
    row_sums_equal = row_sums_ind_agg4 == row_sums_cons

    # Display row sums and the comparison results
    print("Row sums for ind_agg4:\n", row_sums_ind_agg4)
    print("Row sums for cons:\n", row_sums_cons)
    print("Are row sums equal:\n", row_sums_equal)
    # Convert dataframes to vectors
    def vec(x):
        return np.array(x).flatten()  # Converts DataFrame/array to a 1D numeric array

    # Calculate correlations
    cor_agg0 = np.corrcoef(vec(ind_agg0), vec(cons))[0, 1]
    cor_agg1 = np.corrcoef(vec(ind_agg1), vec(cons))[0, 1]
    cor_agg2 = np.corrcoef(vec(ind_agg2), vec(cons))[0, 1]
    cor_agg3 = np.corrcoef(vec(ind_agg3), vec(cons))[0, 1]
    cor_agg4 = np.corrcoef(vec(ind_agg4), vec(cons))[0, 1]

    logger.info(f"Final Correlation between Aggregate and Constraint Dataset: {cor_agg4}")

    # Print correlations
    print("Correlation between ind_agg0 and cons:", cor_agg0)
    print("Correlation between ind_agg1 and cons:", cor_agg1)
    print("Correlation between ind_agg2 and cons:", cor_agg2)
    print("Correlation between ind_agg3 and cons:", cor_agg3)
    print("Correlation between ind_agg4 and cons:", cor_agg4)

        # Initialize an empty DataFrame to store results (equivalent to NULL in R)
    ints_df = pd.DataFrame()

    # Loop through zones to expand the weights and create a DataFrame for each zone
    for i in range(n_zone):
        # Apply `int_trs` to round the weights and obtain the expanded vector
        int_weight1 = int_expand_vector(int_trs(weights4[:, i]))
        
        # Create a DataFrame with the expanded indices from `ind_full`
        data_frame = ind_full.iloc[int_weight1 - 1, :].copy()  # Subtract 1 to adjust for Python's 0-based indexing
        
        # Add a `GEOID` column with the appropriate identifier for each row
        data_frame['GEOID'] = str(geoids.iloc[i]['GEOID'])
        
        # Append `data_frame` to `ints_df` (equivalent to `rbind` in R)
        ints_df = pd.concat([ints_df, data_frame], ignore_index=True)

    # Select specific columns (equivalent to `ints_df <- ints_df[, c(...)]` in R)
    ints_df = ints_df[["GEOID", "AGE", gender_name, "EEDUC", "KINDWORK", twday_name]]
    logger.info(f"Final ints_df population: {ints_df.shape[0]}")
    # Display the resulting DataFrame
    print(ints_df.head())
    return ints_df

In [118]:
def print_results(ints_df):
    print(ints_df['TWDAYS'].value_counts())
    print(ints_df['AGE'].value_counts())
    print(ints_df['EGENID_BIRTH'].value_counts())
    print(ints_df['KINDWORK'].value_counts())
    # Step 1: Data Cleaning and Preparation
    data = ints_df.copy()

    # Check for missing values
    missing_values = data.isna().sum()
    print("Missing values:\n", missing_values)

    # Ensure 'TWDAYS' is consistent and treated as a categorical variable
    data['TWDAYS'] = data['TWDAYS'].str.strip().str.lower()
    # Ensure 'AGE', 'EEDUC', 'EGENID_BIRTH', and 'KINDWORK' are treated as categorical variables
    data['AGE'] = data['AGE'].astype('category')
    data['EEDUC'] = data['EEDUC'].astype('category')
    data['EGENID_BIRTH'] = data['EGENID_BIRTH'].astype('category')
    data['KINDWORK'] = data['KINDWORK'].astype('category')
    # Calculate frequency of telework days
    twday_counts = data['TWDAYS'].value_counts()
    print("Frequency of telework days:\n", twday_counts)

    # Summarize by age group
    age_twday_summary = data.groupby(['AGE', 'TWDAYS']).size().unstack(fill_value=0)
    print("Age and Telework Days Summary:\n", age_twday_summary)

    # Calculate proportions by age group
    age_twday_proportion = data.groupby(['AGE', 'TWDAYS']).size().reset_index(name='Count')
    age_twday_proportion['Proportion'] = age_twday_proportion.groupby('AGE')['Count'].transform(lambda x: x / x.sum())
    age_twday_proportion = age_twday_proportion.pivot(index='AGE', columns='TWDAYS', values='Proportion').fillna(0)
    print("Age and Telework Days Proportion:\n", age_twday_proportion)

    # Summarize by gender
    gender_twday_summary = data.groupby(['EGENID_BIRTH', 'TWDAYS']).size().unstack(fill_value=0)
    print("Gender and Telework Days Summary:\n", gender_twday_summary)

    # Calculate proportions by gender
    gender_twday_proportion = data.groupby(['EGENID_BIRTH', 'TWDAYS']).size().reset_index(name='Count')
    gender_twday_proportion['Proportion'] = gender_twday_proportion.groupby('EGENID_BIRTH')['Count'].transform(lambda x: x / x.sum())
    gender_twday_proportion = gender_twday_proportion.pivot(index='EGENID_BIRTH', columns='TWDAYS', values='Proportion').fillna(0)
    print("Gender and Telework Days Proportion:\n", gender_twday_proportion)

    # Summarize by educational attainment
    education_twday_summary = data.groupby(['EEDUC', 'TWDAYS']).size().unstack(fill_value=0)
    print("Education and Telework Days Summary:\n", education_twday_summary)

    # Calculate proportions by educational attainment
    education_twday_proportion = data.groupby(['EEDUC', 'TWDAYS']).size().reset_index(name='Count')
    education_twday_proportion['Proportion'] = education_twday_proportion.groupby('EEDUC')['Count'].transform(lambda x: x / x.sum())
    education_twday_proportion = education_twday_proportion.pivot(index='EEDUC', columns='TWDAYS', values='Proportion').fillna(0)
    print("Education and Telework Days Proportion:\n", education_twday_proportion)

    # Summarize by kind of work
    kindwork_twday_summary = data.groupby(['KINDWORK', 'TWDAYS']).size().unstack(fill_value=0)
    print("Kindwork and Telework Days Summary:\n", kindwork_twday_summary)

    # Calculate proportions by kind of work
    kindwork_twday_proportion = data.groupby(['KINDWORK', 'TWDAYS']).size().reset_index(name='Count')
    kindwork_twday_proportion['Proportion'] = kindwork_twday_proportion.groupby('KINDWORK')['Count'].transform(lambda x: x / x.sum())
    kindwork_twday_proportion = kindwork_twday_proportion.pivot(index='KINDWORK', columns='TWDAYS', values='Proportion').fillna(0)
    print("Kindwork and Telework Days Proportion:\n", kindwork_twday_proportion)
    # Visualization for telework days by age group
    plt.figure(figsize=(10, 6))
    age_twday_proportion.plot(kind='bar', stacked=True)
    plt.title('Proportion of Telework Days by Age Group')
    plt.xlabel('Age Group')
    plt.ylabel('Proportion of Telework Days')
    plt.xticks(rotation=90)
    plt.legend(title='Telework Days', bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.tight_layout()
    plt.show()

    # Visualization for telework days by gender
    plt.figure(figsize=(10, 6))
    gender_twday_proportion.plot(kind='bar', stacked=True)
    plt.title('Proportion of Telework Days by Gender')
    plt.xlabel('Gender')
    plt.ylabel('Proportion of Telework Days')
    plt.legend(title='Telework Days', bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.tight_layout()
    plt.show()

    # Visualization for telework days by educational attainment
    plt.figure(figsize=(10, 6))
    education_twday_proportion.plot(kind='bar', stacked=True)
    plt.title('Proportion of Telework Days by Educational Attainment')
    plt.xlabel('Educational Attainment')
    plt.ylabel('Proportion of Telework Days')
    plt.xticks(rotation=90)
    plt.legend(title='Telework Days', bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.tight_layout()
    plt.show()

    # Visualization for telework days by kind of work
    plt.figure(figsize=(10, 6))
    kindwork_twday_proportion.plot(kind='bar', stacked=True)
    plt.title('Proportion of Telework Days by Kind of Work')
    plt.xlabel('Kind of Work')
    plt.ylabel('Proportion of Telework Days')
    plt.xticks(rotation=90)
    plt.legend(title='Telework Days', bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.tight_layout()
    plt.show()

In [119]:
# Code to loop over all phase files
# Mechanism to save results from different phases
# Mechanism to combine results from different phases for an entire location
db_path = 'HPS_phases_Data.db'
# Connect to the SQLite database
conn = sqlite3.connect(db_path)
# Query to get all table names
query = "SELECT name FROM sqlite_master WHERE type='table';"
tables_df = pd.read_sql_query(query, conn)
# Display the list of all table names
print("List of all tables:")
print(tables_df['name'].to_list())
tables_df_list = tables_df['name'].to_list()
# Close the database connection
conn.close()

List of all tables:
['Phase_2_HPS_Week13_PUF_CSV(August19_August31)2020_pulse2020_puf_13', 'Phase_2_HPS_Week14_PUF_CSV(September2_September14)2020_pulse2020_puf_14', 'Phase_2_HPS_Week15_PUF_CSV(September16_September28)2020_pulse2020_puf_15', 'Phase_2_HPS_Week16_PUF_CSV(September30_October12)2020_pulse2020_puf_16', 'Phase_2_HPS_Week17_PUF_CSV(October14_October26)2020_pulse2020_puf_17', 'Phase_3_HPS_Week18_PUF_CSV(October28_November9)2020_pulse2020_puf_18', 'Phase_3_HPS_Week19_PUF_CSV(November11_November23)2020_pulse2020_puf_19', 'Phase_3_HPS_Week20_PUF_CSV(November25_December7)2020_pulse2020_puf_20', 'Phase_3_HPS_Week21_PUF_CSV(December9_December21)2020_pulse2020_puf_21', 'Phase_3_HPS_Week22_PUF_CSV(January6_January18)2021_pulse2021_puf_22', 'Phase_3_HPS_Week23_PUF_CSV(January20_February1)2021_pulse2021_puf_23', 'Phase_3_HPS_Week24_PUF_CSV(February3_February15)2021_pulse2021_puf_24', 'Phase_3_HPS_Week25_PUF_CSV(February17_March1)2021_pulse2021_puf_25', 'Phase_3_HPS_Week26_PUF_CSV(March3

In [120]:
# # Edit cell to run IPF on all phase files
# conn = sqlite3.connect(db_path)
# for table in tables_df_list:
#     table_name = table
#     print(table_name)
#     ind_data = pd.read_sql_query(f'SELECT * FROM "{table_name}"', conn)
#     ind_data_new,ind_full_new,new_brks,new_labs,edu_labs,work_labs,gender_name,twday_name = filter_data(MSACODE,STATECODE,ind_data)
#     con_age,con_sex = con_age_func(new_brks,new_labs,statename,countyname)
#     con_edu = con_edu_func(edu_labs,statename,countyname)
#     con_work = con_work_func(work_labs,statename,countyname)
#     cons,con_age,con_sex,con_edu,con_work,geoids = constraint_build(con_age,con_sex,con_edu,con_work)
#     ints_df = ipfmain(cons,con_age,con_sex,con_edu,con_work,ind_data_new,ind_full_new,geoids,gender_name,twday_name)
#     ints_df.to_csv("New Jersey/Middlesex County, NJ/{}_synthetic_population.csv".format(table_name))
# conn.close()

In [121]:
# Edit cell to run IPF on single phase
conn = sqlite3.connect(db_path)
table_name = "Phase_4_2_HPS_Phase4_2Cycle09_PUF_CSV(Aug20_Sep16)2024_hps_04_02_09_puf"
ind_data = pd.read_sql_query(f'SELECT * FROM "{table_name}"', conn)
logger.info(f"Starting processing for {table_name}")
ind_data_new,ind_full_new,new_brks,new_labs,edu_labs,work_labs,gender_name,twday_name = filter_data(MSACODE,STATECODE,ind_data)
con_age,con_sex = con_age_func(new_brks,new_labs,statename,countyname)
con_edu = con_edu_func(edu_labs,statename,countyname)
con_work = con_work_func(work_labs,statename,countyname)
cons,con_age,con_sex,con_edu,con_work,geoids = constraint_build(con_age,con_sex,con_edu,con_work)
ints_df = ipfmain(cons,con_age,con_sex,con_edu,con_work,ind_data_new,ind_full_new,geoids,gender_name,twday_name)
ints_df.to_csv("New Jersey/Middlesex County, NJ/{}_synthetic_population.csv".format(table_name))
conn.close()

          SCRAM  CYCLE  EST_ST  EST_MSA  REGION       HWEIGHT       PWEIGHT  \
39   P090000040      9      34  35620.0       1   2973.320238   2784.759749   
135  P090000136      9      34  35620.0       1   2890.352895  13535.269928   
186  P090000187      9      34  35620.0       1  12640.706480  23678.129353   
217  P090000218      9      34  35620.0       1   3907.462392  10978.982873   
224  P090000225      9      34  35620.0       1   1647.102097   3085.294052   

     TBIRTH_YEAR  ABIRTH_YEAR  RHISPANIC  AHISPANIC  RRACE  ARACE  EEDUC  \
39          1941            2          1          2      1      2      3   
135         1970            2          1          2      1      2      4   
186         1977            2          2          2      1      1      3   
217         1963            2          2          2      1      2      6   
224         1957            2          1          2      1      2      7   

     AEDUC  MS  EGENID_BIRTH  AGENID_BIRTH  SEXUAL_ORIENTATION_RV  \